# Running Noah-MP Component Model Under UFS Weather Model


This tutorial aims to give brief information about running the Noah-MP land model as a component under the UFS Weather Model. In this case, we will leverage the use of Docker containers to create an isolated run environment.


## Docker Container


A [Docker](https://www.docker.com) container image is a lightweight, standalone, executable package of software that includes everything needed to run an application: code, runtime, system tools, system libraries and settings.


For this tutorial, we build a special Docker container that includes following components,


- Core development tools such as GNU compiler that are provided by Ubuntu 22.04
- [Spack-stack](https://github.com/JCSDA/spack-stack)


To run the Docker container, users need to install [Docker Desktop](https://www.docker.com/products/docker-desktop/) which provides secure, out-of-the-box containerization software to build, share, and run applications.


### Spack-stack


Spack-stack is a framework for installing software libraries to support NOAA's Unified Forecast System (UFS) applications and the Joint Effort for Data assimilation Integration (JEDI) coupled to several Earth system prediction models (MPAS, NEPTUNE, UM, FV3, GEOS, UFS). It allows us to build all the dependencies in a flexible and easy way. More information about porting UFS Weather Model can be found in [here](https://spack-stack.readthedocs.io/en/latest/).


The Docker container that will be used in this tutorial includes Spack-stack version 1.5.1 to install UFS Weather Model dependencies like libraries, tools etc.


## UFS Weather Model Noah-MP Regression Tests


The UFS Weather Model includes the following RTs to test Noah-MP land component model functionality.


| Test | Description | Active Components |
|---------------------------|--------------------------------------------------------------------------------|------------------------|
| datm_cdeps_lnd_gswp3 | Noah-MP model forced by GSWP3 dataset | Noah-MP, CDEPS , CMEPS |
| datm_cdeps_lnd_era5 | Noah-MP model forced by ECMWF's ERA5 dataset | Noah-MP, CDEPS , CMEPS |
| datm_cdeps_lnd_era5_rst | Same with datm_cdeps_lnd_era5 but tests restart functionality | Noah-MP, CDEPS , CMEPS |
| control_p8_atmlnd_sbs | Noah-MP model forced by active atmosphere model, side-by-side, one-way coupling | Noah-MP, FV3ATM, CMEPS |
| control_p8_atmlnd | Noah-MP model forced by active atmosphere model, two-way coupling | Noah-MP, FV3ATM, CMEPS |
| control_restart_p8_atmlnd | Same with control_p8_atmlnd but tests restart functionality | Noah-MP, FV3ATM, CMEPS |


In this tutorial, the Docker container is only able to run `datm_cdeps_lnd_era5` and `datm_cdeps_lnd_era5_rst` tests due to lack of required input files for other configurations and their computational requirements.


## Running datm_cdeps_lnd_era5 Test


The **datm_cdeps_lnd_era5** test is a global model configuration in C96 (~ 1 degree) resolution and uses a cubed-sphere grid. The simulation starts on 2011-01-01 and performs a 12-hours simulation.


1. Run Docker container


```shell
$ docker run --rm --platform linux/amd64 --hostname=linux -it turuncu/ams_ufs:latest
```


> **_NOTE:_** The size of the container is large (~30 GB) due to the number of dependencies required by the UFS Weather Model and it could take time to pull and extract all the content. So, be patient.


3. Clone UFS Weather Model


```shell
[root@linux]$ cd /opt
[root@linux]$ git clone -b feature/ams_course --recursive https://github.com/uturuncoglu/ufs-weather-model.git
```


> **_NOTE:_** The UFS Weather Model used in here is a development fork, which is customized to run under Docker container. Please use official repository of UFS Weather Model for future work.


4. Run regression test


```shell
[root@linux]$ cd ufs-weather-model/tests
[root@linux]$ ./rt.sh -a debug -k -n datm_cdeps_lnd_era5 gnu
```


> **_NOTE:_** Since datm_cdeps_lnd_era5 test is a global configuration, the regression tests use 6 MPI processes (the minimal number of processes that can be used in global application, one process for each tile in the cubed-sphere grid). If the Docker container does not have access to 6 cores, the case could still run but it would be slow.


5. Creating visualization using NCL


Find your run directory under `/opt/scratch/root/FV3_RT/`. Under the run directory following commands can be used to copy required NCL (NCAR Command Language) visualization scripts.


```shell
[root@linux]$ cd [RUN DIRECTORY]
[root@linux]$ cp /opt/tools/*.ncl .
[root@linux]$ ncl plot_cubed.ncl
[root@linux]$ ncl plot_cubed_ts.ncl
```


> **_NOTE:_** Replace [RUN DIRECTORY] with the actual one.


To copy the plots to the local machine `docker cp` command can be used. To find the container ID, following commands can be run in the host/physical machine


```shell
[user@local]$ docker ps


# example
CONTAINER ID IMAGE COMMAND CREATED STATUS PORTS NAMES
e796af101646 turuncu/ams_ufs:latest "/entrypoint.sh bash" 25 minutes ago Up 25 minutes silly_blackwell
```


Then, files can be copied with following command,


```shell
[user@local]$ docker cp [CONTAINER ID]:[RUN DIRECTORY]/plot_cubed_01-01-2011_12:00_hflx.pdf .
[user@local]$ docker cp [CONTAINER ID]:[RUN DIRECTORY]/plot_cubed_ts_evap.pdf .
```


> **_NOTE:_** Replace [RUN DIRECTORY] and [CONTAINER ID] with actual ones.

